In [130]:
from drlqap.taskgenerators import generators
import scipy.optimize
from drlqap.simplesolver import solve_qap_backtracking, solve_qap_maxgreedy, solve_qap_faq
from drlqap.gurobi import solve_qap_gurobi
import numpy as np
from drlqap.qap import QAP

In [2]:
# change to project root
%cd ..

/home/jupyter-tim/ba-tim


In [3]:
generators

{'small_random_graphs': <drlqap.taskgenerators.RandomWeightsTaskGenerator at 0x7f07604f6760>,
 'medium_random_graphs': <drlqap.taskgenerators.RandomWeightsTaskGenerator at 0x7f06c2069220>,
 'qaplib_bur26a': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f06c20692e0>,
 'qaplib_bur26a_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f06c2069340>,
 'small_fixed': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f06c20693d0>,
 'triangle': <drlqap.taskgenerators.SingleTask at 0x7f06c2069460>,
 'qaplib_all_bur': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f06c2069550>,
 'qaplib_sko_42_64_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f06c20694f0>,
 'qaplib_all_bur_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f06c2069610>,
 'qaplib_all_chr_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f06c2069670>,
 'qaplib_all_esc_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7f06c20696d0>,
 'qaplib_all_had_nor

In [82]:
qap = generators['medium_random_graphs'].sample()

In [5]:
solve_qap_maxgreedy(qap)

(2.3655079779637163,
 [10,
  12,
  14,
  25,
  13,
  5,
  1,
  6,
  11,
  2,
  4,
  8,
  7,
  17,
  3,
  20,
  0,
  19,
  18,
  24,
  9,
  22,
  16,
  23,
  21,
  15])

In [95]:
solve_qap_faq(qap)

(57.700447,
 array([ 7,  8,  5, 10,  2,  1,  9,  4,  0, 14, 15, 13, 11,  6, 12,  3]))

In [7]:
#solve_qap_gurobi(qap)

In [8]:
%timeit solve_qap_faq(qap)

7.52 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
!ls runs

a2c_mediumrandoms		    dqn_dense_ms_ec_eps0_smallrandoms
a2c_ms100x_mediumrandoms	    dqn_linkedqap_smallrandoms
dqn_dense_ms_ec_eps0_mediumrandoms  reinforce_ms100x_mediumrandoms
dqn_dense_ms_ec_eps0_rni_study	    reinforce_ms100x_smallrandoms


In [31]:
!ls runs/dqn_dense_ms_ec_eps0_smallrandoms

lr1e-4_s1  lr1e-4_s4  lr1e-5_s3  lr3e-4_s2  lr5e-4_s1  lr5e-4_s4
lr1e-4_s2  lr1e-5_s1  lr1e-5_s4  lr3e-4_s3  lr5e-4_s2
lr1e-4_s3  lr1e-5_s2  lr3e-4_s1  lr3e-4_s4  lr5e-4_s3


In [10]:
from drlqap.evaltools import load_checkpoints
from pathlib import Path

In [219]:
agents = load_checkpoints(Path('runs/a2c_ms100x_mediumrandoms/lr4e-5_s3'))

runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_0.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_1000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_2000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_3000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_4000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_5000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_6000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_7000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_8000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_9000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_10000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_11000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_12000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_13000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_14000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_15000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_

In [220]:
agents[-1].solve(qap)

(61.25126391649246, [7, 8, 2, 3, 10, 14, 0, 13, 12, 5, 11, 9, 4, 6, 15, 1])

## Average results

In [221]:
def evaluate(solver, problem, samples):
    values = []
    for i in range(samples):
        qap = generators[problem].sample()
        v, _ = solver(qap)
        values.append(v)
    return np.mean(values)

def evaluate_set(solver, problem):
    values = []
    test_set = generators[problem].test_set()
    for qap in test_set:
        v, _ = solver(qap)
        values.append(v)
    return np.mean(values)

In [222]:
evaluate(solve_qap_faq, 'small_random_graphs', 1000)

11.430882

In [223]:
evaluate(agents[-1].solve, 'small_random_graphs', 500)

11.643129579840053

In [224]:
evaluate(solve_qap_faq, 'medium_random_graphs', 1000)

49.72502

In [225]:
evaluate(agents[-1].solve, 'medium_random_graphs', 500)

52.78447303925222

In [226]:
evaluate_set(solve_qap_faq, 'qaplib_all_64_normalized')

31925654.0

In [227]:
evaluate_set(agents[-1].solve, 'qaplib_all_64_normalized')

55562342.18585188

In [228]:
evaluate_set(solve_qap_faq, 'qaplib_tai35a_normalized')

2514002.0

In [229]:
evaluate_set(agents[-1].solve, 'qaplib_tai35a_normalized')

2755434.0

In [230]:
evaluate_set(agents[-1].solve, 'qaplib_tai35a_normalized')

2755434.0

## Qaplib

In [104]:
qaplib = generators["qaplib_all_64_normalized"].test_set()
qaplib_by_name = {q.name: q for q in qaplib}

In [112]:
agents[-1].solve(qaplib_by_name["bur26c"])

(6153352.0,
 [2,
  0,
  19,
  13,
  18,
  21,
  23,
  10,
  25,
  24,
  15,
  14,
  4,
  8,
  7,
  22,
  6,
  5,
  3,
  1,
  20,
  12,
  17,
  11,
  16,
  9])

In [164]:
solve_qap_faq(qaplib_by_name["bur26c"])

(5418795.0,
 array([22,  2,  6, 20, 12, 10, 21, 19,  7, 11,  4,  8,  0, 18,  5,  3, 14,
        13, 17, 16, 24, 23, 25,  9,  1, 15]))

# Asymmetry

In [139]:
def mirror(qap):
    a = qap.A
    b = qap.B
    return QAP(b, a, qap.linear_costs.transpose(0,1), 0)
    
def mirror_assignment(assignment):
    out = [None] * len(assignment)
    for i, j in enumerate(assignment):
        out[j] = i
    return out

In [140]:
qap_mirrored = mirror(qap)

In [141]:
qap.compute_value([12, 13, 2, 10, 1, 14, 4, 7, 6, 8, 3, 11, 9, 0, 5, 15])

tensor(61.8118)

In [144]:
qap_mirrored.compute_value(mirror_assignment([12, 13, 2, 10, 1, 14, 4, 7, 6, 8, 3, 11, 9, 0, 5, 15]))

tensor(61.8118)

In [147]:
agents[-1].solve(qap)

(61.25126391649246, [7, 8, 2, 3, 10, 14, 0, 13, 12, 5, 11, 9, 4, 6, 15, 1])

In [148]:
agents[-1].solve(qap_mirrored)

(58.26400242745876, [0, 2, 6, 10, 7, 1, 4, 11, 15, 13, 3, 12, 14, 8, 5, 9])

In [165]:
evaluate_set(lambda qap: solve_qap_faq(mirror(qap)), 'qaplib_all_64_normalized')

31625270.0

In [176]:
evaluate_set(lambda qap: agents[-1].solve(mirror(qap)), 'qaplib_all_64_normalized')

36323050.070392564

In [157]:
evaluate(lambda qap: agents[-1].solve(mirror(qap)), 'medium_random_graphs', 500)

52.60779935359853

In [158]:
evaluate(lambda qap: agents[-1].solve(mirror(qap)), 'small_random_graphs', 500)

11.754977045599487

In [163]:
agents[-1].solve(mirror(qaplib_by_name["bur26c"]))

(5704272.0,
 [8,
  3,
  6,
  11,
  25,
  4,
  5,
  18,
  7,
  19,
  23,
  17,
  2,
  10,
  16,
  15,
  24,
  13,
  1,
  12,
  21,
  22,
  0,
  20,
  14,
  9])

In [213]:
agents = load_checkpoints(Path('runs/'))

runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_0.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_1000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_2000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_3000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_4000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_5000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_6000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_7000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_8000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_9000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_10000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_11000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_12000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_13000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_14000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_15000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_

In [216]:
evaluate_set(agents[-1].solve, 'qaplib_all_bur_normalized')

5.099721514828161

In [217]:
evaluate_set(lambda qap: agents[-1].solve(mirror(qap)), 'qaplib_all_bur_normalized')

3.899634671169224